In [ ]:
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
import plotly.graph_objects as go
from plotting import CandlePlot
from data import Data
pd.set_option("display.max_columns", None)

In [ ]:
data_path = '../data/'
out_path = 'D:/Trading/ml4t-data/grid/'
instruments = "../data/instruments.json"
d = Data(source="../data/EUR_USD_M5.pkl", ticker='EUR_USD', instruments=instruments)

In [ ]:
our_cols = ['time', 'mid_c']

add_cols = dict(
            pip_returns=float,
            move_1_begin=int,
            move_1_pips=float,
            move_1_direction=object,
            move_2_begin=int,
            move_2_pips=float,
            move_2_direction=object,
            move_3_begin=int,
            move_3_pips=float,
            move_3_direction=object,
            move_4_begin=int,
            move_4_pips=float,
            move_4_direction=object,
            move_5_begin=int,
            move_5_pips=float,
            move_5_direction=object,
            move_6_begin=int,
            move_6_pips=float,
            move_6_direction=object,
            move_7_begin=int,
            move_7_pips=float,
            move_7_direction=object,
            move_8_begin=int,
            move_8_pips=float,
            move_8_direction=object,
            move_9_begin=int,
            move_9_pips=float,
            move_9_direction=object,
            move_10_begin=int,
            move_10_pips=float,
            move_10_direction=object,
            move_11_begin=int,
            move_11_pips=float,
            move_11_direction=object,
            move_12_begin=int,
            move_12_pips=float,
            move_12_direction=object,
            move_13_begin=int,
            move_13_pips=float,
            move_13_direction=object,
            move_14_begin=int,
            move_14_pips=float,
            move_14_direction=object,
            move_15_begin=int,
            move_15_pips=float,
            move_15_direction=object,
            move_16_begin=int,
            move_16_pips=float,
            move_16_direction=object,
            move_17_begin=int,
            move_17_pips=float,
            move_17_direction=object,
            move_18_begin=int,
            move_18_pips=float,
            move_18_direction=object,
            move_19_begin=int,
            move_19_pips=float,
            move_19_direction=object,
            move_20_begin=int,
            move_20_pips=float,
            move_20_direction=object,
            moves_trend=object
        )

candles = d.datalen
d.prepare_fast_data(name='ana2', start=0, end=candles, cols=our_cols, add_cols=add_cols)

In [ ]:
MOVE_COUNT = 20
PIPS = 10

In [ ]:
def calc_pip_returns(price: np.ndarray) -> np.ndarray:
    next_price = np.append(price[1:], np.nan)
    return (next_price - price) * pow(10, 4)

def calc_pip_returns(price: pd.Series) -> pd.Series:
    prev_price = price.shift(1)
    return (price - prev_price) * pow(10, 4)
# d.update_fdata('pip_returns', value=calc_pip_returns(d.fdata('mid_c')))
d.df['ana2']['pip_returns'] = calc_pip_returns(d.df['ana2']['mid_c'])

In [ ]:
d.df['ana2'].head()

In [ ]:
d.df['ana2']['pip_returns'].describe(percentiles=[0.0001, 0.001, 0.005, .01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99, .995, .999, .9999])

In [ ]:
1.662000e+02 - 166

In [ ]:
d.df['ana2']['pip_returns'].max()

In [ ]:
# d.update_fdata('pip_returns', index=1, value=0)

In [ ]:
d.df['ana2'].head()

In [ ]:
# d.fastdf

In [ ]:
# d.update_fdata('pip_returns', index=2, value=10)

In [ ]:
# d.fdata('pip_returns', 2)

In [ ]:
d.df['ana2']

In [ ]:
for i in tqdm(range(candles), desc=" Processing... "):
    pivot = i
    for j in range(MOVE_COUNT):
        # print(i, j, pivot)
        if pivot + 1 > candles:
            break
        close_price, forward_close_prices = d.fdata('mid_c', index=pivot), d.fdata('mid_c', index=pivot+1, rows=5000)
        pip_returns = np.round((forward_close_prices - close_price) * pow(10, 4), 2)
        d.update_fdata(f'move_{j+1}_begin', i, pivot+1)
        # target = pip_returns > PIPS or pip_returns < -PIPS
        targetup = pip_returns >= PIPS
        targetdown = pip_returns <= -PIPS
        try:
            pivotup = np.where(targetup)[0][0]
        except:
            pivotup = -1
        try:
            pivotdown = np.where(targetdown)[0][0]
        except:
            pivotdown = -1
        if pivotup == -1 and pivotdown == -1:
            break
        elif pivotup >= 0 and pivotdown >= 0:
            returns_pivot = min(pivotup, pivotdown)
        elif pivotup == -1:
            returns_pivot = pivotdown
        elif pivotdown == -1:
            returns_pivot = pivotup

        # print(i, pivot, pivotup, pivotdown, pip_returns[pivot])
        # try:
        d.update_fdata(f'move_{j+1}_pips', i, pip_returns[returns_pivot])
        d.update_fdata(f'move_{j+1}_direction', i, '1' if pip_returns[returns_pivot] > 0 else '0')
        # except:
        #     passp
        pivot = pivot + returns_pivot + 1 # Update pivot to close timestamp: Close means when PIPS target met
            

In [ ]:
d.df['ana2']['moves_trend'].fillna("'", inplace=True)
for i in range(MOVE_COUNT):
    d.df['ana2']['moves_trend'] = d.df['ana2']['moves_trend'] + d.df['ana2'][f'move_{i+1}_direction'].astype(str)

In [ ]:
d.df['ana2']['moves_trend'] = d.df['ana2']['moves_trend'].str.replace('nan', '')

In [ ]:
d.df['ana2'].head()

In [ ]:
d.df['ana2'].info()

In [ ]:
d.df['ana2'].to_csv('D:/Trading/ml4t-data/test5.csv')

In [ ]:
# cp = CandlePlot(d.df['ana2'][['time','mid_c']][41440:415258], candles=False)
# cp.show_plot()
cp = CandlePlot(d.df['ana2'].iloc[41440:415258], candles=False)
# cp.create_candle_fig()
cp.show_plot(line_traces=['mid_c'])